In [1]:
# Machine Learning with Tensorflow
# get data here http://www.cs.toronto.edu/~kriz/cifar.html
# run python tensorflow 1.x code
# https://github.com/BinRoot/TensorFlow-Book.git



In [2]:
# imports
import tensorflow as tf
import pickle
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, MaxPool2D
from keras.utils import to_categorical
from keras import models, layers
%matplotlib inline 

Using TensorFlow backend.


In [3]:
def unpickle(file):
    fo = open(file, 'rb')
    this_dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return this_dict

In [4]:
# cleaning the data
def clean(data):
    imgs = data.reshape(data.shape[0], 3, 32, 32)
    grayscale_imgs = imgs.mean(1)
    cropped_imgs = grayscale_imgs[:, 4:28, 4:28]
    img_data = cropped_imgs.reshape(data.shape[0], -1)
    img_size = np.shape(img_data)[1]
    means = np.mean(img_data, axis=1)
    meansT = means.reshape(len(means), 1)
    stds = np.std(img_data, axis=1)
    stdsT = stds.reshape(len(stds), 1)
    adj_stds = np.maximum(stdsT, 1.0 / np.sqrt(img_size))
    normalized = (img_data - meansT) / adj_stds
    return normalized

In [5]:
def read_data(directory):
    names = unpickle('{}/batches.meta'.format(directory))['label_names']
    print('names', names)
    data, labels = [], []
    for i in range(1, 6):
        filename = '{}/data_batch_{}'.format(directory, i)
        batch_data = unpickle(filename)
        if len(data) > 0: 
            data = np.vstack((data, batch_data['data']))
            labels = np.hstack((labels, batch_data['labels'])
                              )        
        else:
            data = batch_data['data']
            labels = batch_data['labels']
    print(np.shape(data), np.shape(labels))
    data = clean(data) 
    data = data.astype(np.float32)
    return names, data, labels


In [6]:
learning_rate = 0.001

names, data, labels = \
    read_data(r'D:\Documents\1_Projects\CRSIP_ML\data\cifar-10-batches-py')

names ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(50000, 3072) (50000,)


In [7]:
data


array([[-0.6937296 , -0.71540314, -0.686505  , ..., -0.08686984,
        -0.29638094, -0.18801312],
       [ 1.3697398 ,  1.2528099 ,  0.72940975, ..., -0.68488437,
        -1.1860123 , -1.7038443 ],
       [ 1.7809031 ,  1.7809031 ,  1.7809031 , ..., -0.59378237,
        -0.58099234, -0.59378237],
       ...,
       [ 1.12856   ,  1.1333368 ,  1.1715513 , ..., -0.50034004,
        -0.6388682 , -0.8012805 ],
       [ 1.3223659 ,  1.3432754 ,  1.378125  , ..., -0.8173882 ,
        -1.2704306 , -3.382305  ],
       [ 0.09468836,  0.33273417,  0.56377864, ...,  0.01767354,
        -0.19236688, -0.03133589]], dtype=float32)

In [19]:
data.shape
# data_reshaped = tf.reshape(data, shape=[-1, 24, 24, 1])
data_reshaped = data.reshape(data.shape[0], 24, 24, 1)
data_reshaped.shape


(50000, 24, 24, 1)

In [20]:
input_shape  = data_reshaped.shape
input_shape

(50000, 24, 24, 1)

In [32]:
model = models.Sequential()
model.add(layers.Conv2D(16, 3, activation='relu', input_shape=(24, 24, 1)))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPool2D(padding='same'))
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPool2D(padding='same'))
model.add(layers.Conv2D(128, 1, activation='relu'))
model.add(layers.MaxPool2D(padding='same'))
model.add(layers.Conv2D(256, 1, activation='relu'))

#model.add(layers.MaxPool2D())
#model.add(layers.Flatten())  # This layer is not compatible with ARM-NN
#model.add(layers.Reshape([256])) # still implements as strided slice, not compatible
#model.add(layers.Lambda(lambda x: K.reshape(x, [-1, 256])))  # reshape from/to 4D tensor not supported
#model.add(layers.Dense(256, activation='relu'))

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))


model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [33]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 22, 22, 16)        160       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 11, 11, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 9, 9, 32)          4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 3, 64)          18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 2, 2, 128)        

In [34]:
# model.compile(optimizer='adam', metrics=['accuracy'], loss='sparse_categorical_crossentropy')
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')


In [35]:
labels_cat = to_categorical(labels)
labels_cat.shape

(50000, 10)

In [36]:
labels_cat[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [37]:
model.fit(data_reshaped, labels_cat, epochs=20, batch_size=64)


Epoch 1/20
50000/50000 [==============================] - 4s 85us/step - loss: 1.8064 - accuracy: 0.3352
Epoch 2/20
50000/50000 [==============================] - 4s 77us/step - loss: 1.4469 - accuracy: 0.4881
Epoch 3/20
50000/50000 [==============================] - 4s 78us/step - loss: 1.2917 - accuracy: 0.5485
Epoch 4/20
50000/50000 [==============================] - 4s 85us/step - loss: 1.1795 - accuracy: 0.5896
Epoch 5/20
50000/50000 [==============================] - 4s 83us/step - loss: 1.0981 - accuracy: 0.6197
Epoch 6/20
50000/50000 [==============================] - 4s 87us/step - loss: 1.0393 - accuracy: 0.6390
Epoch 7/20
50000/50000 [==============================] - 4s 81us/step - loss: 0.9884 - accuracy: 0.6578
Epoch 8/20
50000/50000 [==============================] - 4s 83us/step - loss: 0.9499 - accuracy: 0.6723
Epoch 9/20
50000/50000 [==============================] - 4s 82us/step - loss: 0.9065 - accuracy: 0.6865
Epoch 10/20
50000/50000 [==============================